In [2]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess
from monai.networks.blocks import Warp
import torch

In [3]:
#monai.metrics.DiceMetric
monai.config.deviceconfig.print_config()

MONAI version: 0.9.1
Numpy version: 1.21.5
Pytorch version: 1.11.0+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 356d2d2f41b473f588899d705bbc682308cee52c
MONAI __file__: /home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.10
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.10.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.12.0+cu113
tqdm version: 4.63.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.1
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
import torchmetrics
import pandas as pd

In [8]:
map6_names = ["bg", 
              "hy", 
              "mb", 
              "hpf", 
              "str", 
              "ctx", 
              "gp", 
              "th", 
              "cb",
              "olf",
              "bs",
             ]
svbp_names = ["bg", 
              "olfactory bulb", 
              "cortex", 
              "hippocampal formation", 
              "striatum", 
              "globus pallidus", 
              "hypothalamus", 
              "thalamus", 
              "coliculi",
              "cerebelum",
             ]

In [19]:
# Before convert

In [11]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels_map6 = "atlas_gin_map6.nii.gz"
atlas_labels_svbp = "atlas_gin_svbp.nii.gz"

deep_labels = sorted(glob(os.path.join('output', 'GIN', 'test', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable.nii.gz")))

#dicemetric = monai.metrics.DiceMetric(num_classes=11)
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
#dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
#dice_metric(y_pred=val_outputs, y=val_labels)

map6_values_deep = [0,0,0,0,0,0,0,0,0,0,0]
map6_values_ants = [0,0,0,0,0,0,0,0,0,0,0]
svbp_values_deep = [0,0,0,0,0,0,0,0,0,0,0]
svbp_values_ants = [0,0,0,0,0,0,0,0,0,0,0]


mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)

def find_map6_svbp_mapping(index, data):
    save_k = -1
    save_l = -1
    for k in range(len(data)):
        if data["map6nifti"][k] == j:
            save_k = k
        if data["svbpnifti"][k] == j:
            save_l = k
    return save_k, save_l

    
map6 = torch.from_numpy(nib.load(atlas_labels_map6).get_fdata().ravel()).type('torch.LongTensor')
svbp = torch.from_numpy(nib.load(atlas_labels_svbp).get_fdata().ravel()).type('torch.LongTensor')
svbp[np.where(svbp < 0)] = 0
map6_count = 0
svbp_count = 0
for i, qsdfgg in enumerate(ants_labels):  
    print(ants_labels[i])
    print(deep_labels[i])
    deep = torch.from_numpy(nib.load(deep_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep[np.where(deep > 10)] = 0
    ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ants[np.where(ants > 10)] = 0   
    
    if "Labels/M" in deep_labels[i]:
        map6_count += 1
        deep_values = dicemetric(deep,map6)
        ants_values = dicemetric(ants,map6)    
        for j in range(len(deep_values)):
            map6_values_deep[j] += deep_values[j].item()
            map6_values_ants[j] += ants_values[j].item()
        print(deep_values)
        print(ants_values)
    else:
        svbp_count += 1
        deep_values = dicemetric(deep,svbp)
        ants_values = dicemetric(ants,svbp)    
        for j in range(len(deep_values)):
            svbp_values_deep[j] += deep_values[j].item()
            svbp_values_ants[j] += ants_values[j].item()  
        print(deep_values)
        print(ants_values) 
    print('-'*10)

for j in range(len(deep_values)):
    map6_values_deep[j] /= map6_count
    map6_values_ants[j] /= map6_count
    svbp_values_deep[j] /= svbp_count
    svbp_values_ants[j] /= svbp_count
    


print(map6_values_deep)
print(map6_values_ants)
print('-'*20)
print(svbp_values_deep)
print(svbp_values_ants)
    
    

dataset3/GIN/Labels/653_N4_o_128_ras_norm_id_affine_deformable.nii.gz
output/GIN/test/Labels/653_N4_o_128_ras_norm_id_affine_dl_labels.nii.gz
tensor([0.9888, 0.6593, 0.9020, 0.9012, 0.8955, 0.7674, 0.8357, 0.8947, 0.7851,
        0.8903,    nan])
tensor([0.9862, 0.6327, 0.8972, 0.8033, 0.8616, 0.6043, 0.7265, 0.7482, 0.4498,
        0.8318,    nan])
----------
dataset3/GIN/Labels/655_N4_o_128_ras_norm_id_affine_deformable.nii.gz
output/GIN/test/Labels/655_N4_o_128_ras_norm_id_affine_dl_labels.nii.gz
tensor([0.9873, 0.6278, 0.8857, 0.8954, 0.8220, 0.7097, 0.8169, 0.8546, 0.7404,
        0.8823,    nan])
tensor([0.9846, 0.6218, 0.8765, 0.8061, 0.8034, 0.6888, 0.7160, 0.7187, 0.4448,
        0.8259,    nan])
----------
dataset3/GIN/Labels/656_N4_o_128_ras_norm_id_affine_deformable.nii.gz
output/GIN/test/Labels/656_N4_o_128_ras_norm_id_affine_dl_labels.nii.gz
tensor([0.9881, 0.6408, 0.8986, 0.8776, 0.8736, 0.7300, 0.8393, 0.8876, 0.7769,
        0.8801,    nan])
tensor([0.9849, 0.5826, 0.8

tensor([0.9864, 0.7926, 0.7321, 0.9050, 0.8951, 0.8728, 0.6063, 0.8740, 0.9160,
        0.6267, 0.8339])
tensor([0.9871, 0.7353, 0.6605, 0.8598, 0.8926, 0.8954, 0.5502, 0.8324, 0.9154,
        0.7222, 0.8258])
----------
dataset3/GIN/Labels/M877_N4_o_128_ras_norm_id_affine_deformable.nii.gz
output/GIN/test/Labels/M877_N4_o_128_ras_norm_id_affine_dl_labels.nii.gz
tensor([0.9857, 0.8106, 0.0000, 0.8997, 0.9006, 0.8799, 0.6104, 0.8708, 0.9102,
        0.6451, 0.8286])
tensor([0.9852, 0.7746, 0.0000, 0.8318, 0.8721, 0.8856, 0.4047, 0.8212, 0.8947,
        0.7082, 0.7860])
----------
dataset3/GIN/Labels/M886_N4_o_128_ras_norm_id_affine_deformable.nii.gz
output/GIN/test/Labels/M886_N4_o_128_ras_norm_id_affine_dl_labels.nii.gz
tensor([0.9861, 0.7334, 0.7373, 0.8984, 0.8839, 0.8767, 0.6619, 0.8777, 0.9234,
        0.5928, 0.8101])
tensor([0.9855, 0.6443, 0.6432, 0.8277, 0.8463, 0.8730, 0.4333, 0.7922, 0.9110,
        0.7066, 0.8140])
----------
dataset3/GIN/Labels/M8876_N4_o_128_ras_norm_id_af

In [12]:
for i, name in enumerate(map6_names):
    print((name + ": ants={:.4f} | deep={:.4f}").format(map6_values_ants[i], map6_values_deep[i]))
print('-'*20)
for i, name in enumerate(svbp_names):
    print((name + ": ants={:.4f} | deep={:.4f}").format(svbp_values_ants[i], svbp_values_deep[i]))

bg: ants=0.9860 | deep=0.9864
hypothalamus: ants=0.7432 | deep=0.7988
coliculi: ants=0.5655 | deep=0.6921
hippocampal formation: ants=0.8328 | deep=0.8980
striatum: ants=0.8776 | deep=0.9003
cortex: ants=0.8808 | deep=0.8761
globus pallidus: ants=0.4671 | deep=0.6639
thalamus: ants=0.8152 | deep=0.8724
cerebelum: ants=0.8966 | deep=0.9146
olfactory bulb: ants=0.7266 | deep=0.6265
brainstem: ants=0.7972 | deep=0.8159
--------------------
bg: ants=0.9855 | deep=0.9880
olfactory bulb: ants=0.6322 | deep=0.6613
cortex: ants=0.8849 | deep=0.8940
hippocampal formation: ants=0.7933 | deep=0.8854
striatum: ants=0.8423 | deep=0.8788
globus pallidus: ants=0.6240 | deep=0.7005
hypothalamus: ants=0.7080 | deep=0.8074
thalamus: ants=0.7516 | deep=0.8605
coliculi: ants=0.4970 | deep=0.7907
cerebelum: ants=0.8298 | deep=0.8755


In [18]:
# After SVBP convert to MAP6

In [13]:
atlas_labels_map6 = "atlas_gin_map6.nii.gz"
atlas_labels_svbp = "atlas_gin_svbp.nii.gz"

deep_labels = sorted(glob(os.path.join('output', 'GIN', 'test_convert', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))

#dicemetric = monai.metrics.DiceMetric(num_classes=11)
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
#dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
#dice_metric(y_pred=val_outputs, y=val_labels)

values_deep = [0,0,0,0,0,0,0,0,0,0,0]
values_ants = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
map6 = torch.from_numpy(nib.load(atlas_labels_map6).get_fdata().ravel()).type('torch.LongTensor')
count = 0

for i, _ in enumerate(ants_labels):  
    count += 1
    print(ants_labels[i])
    print(deep_labels[i])
    deep = torch.from_numpy(nib.load(deep_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep[np.where(deep > 10)] = 0
    ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ants[np.where(ants > 10)] = 0   
    
    deep_values = dicemetric(deep,map6)
    ants_values = dicemetric(ants,map6)    
    for j in range(len(deep_values)):
        values_deep[j] += deep_values[j].item()
        values_ants[j] += ants_values[j].item()
    print(deep_values)
    print(ants_values)
    print('-'*10)

for j in range(len(deep_values)):
    values_deep[j] /= count
    values_ants[j] /= count
    


print(values_deep)
print(values_ants)
print('-'*20) 
    

dataset3/GIN/Labels/653_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/653_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9843, 0.8357, 0.7851, 0.9012, 0.8955, 0.9020, 0.7674, 0.8947, 0.8903,
        0.6593, 0.0000])
tensor([0.9815, 0.7265, 0.4498, 0.8033, 0.8616, 0.8972, 0.6043, 0.7482, 0.8318,
        0.6327, 0.0000])
----------
dataset3/GIN/Labels/655_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/655_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9827, 0.8169, 0.7404, 0.8954, 0.8220, 0.8857, 0.7097, 0.8546, 0.8823,
        0.6278, 0.0000])
tensor([0.9799, 0.7160, 0.4448, 0.8061, 0.8034, 0.8765, 0.6888, 0.7187, 0.8259,
        0.6218, 0.0000])
----------
dataset3/GIN/Labels/656_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/656_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9835, 0.8393, 0.7769, 0.8776, 0.8736, 0.8986, 0.7300, 0.8876, 0.8801,
        0.640

tensor([0.9869, 0.8001, 0.7969, 0.9024, 0.9113, 0.8793, 0.7543, 0.8601, 0.9180,
        0.5902, 0.8261])
tensor([0.9872, 0.7444, 0.6906, 0.8203, 0.8837, 0.8849, 0.6465, 0.7972, 0.8990,
        0.7823, 0.8075])
----------
dataset3/GIN/Labels/M875_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/M875_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9864, 0.7926, 0.7321, 0.9050, 0.8951, 0.8728, 0.6063, 0.8740, 0.9160,
        0.6267, 0.8339])
tensor([0.9871, 0.7353, 0.6605, 0.8598, 0.8926, 0.8954, 0.5502, 0.8324, 0.9154,
        0.7222, 0.8258])
----------
dataset3/GIN/Labels/M877_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/M877_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9857, 0.8106, 0.0000, 0.8997, 0.9006, 0.8799, 0.6104, 0.8708, 0.9102,
        0.6451, 0.8286])
tensor([0.9852, 0.7746, 0.0000, 0.8318, 0.8721, 0.8856, 0.4047, 0.8212, 0.8947,
        0.7082, 0.7860])
----------
dataset3/GIN/Labels/M88

In [17]:
for i, name in enumerate(map6_names):
    print((name + ": ants={:.4f} | deep={:.4f}").format(values_ants[i], values_deep[i]))

bg: ants=0.9830 | deep=0.9846
hypothalamus: ants=0.7224 | deep=0.8039
coliculi: ants=0.5249 | deep=0.7506
hippocampal formation: ants=0.8094 | deep=0.8905
striatum: ants=0.8567 | deep=0.8876
cortex: ants=0.8832 | deep=0.8867
globus pallidus: ants=0.5601 | deep=0.6856
thalamus: ants=0.7775 | deep=0.8654
cerebelum: ants=0.8570 | deep=0.8914
olfactory bulb: ants=0.6707 | deep=0.6472
brainstem: ants=0.3248 | deep=0.3324


In [5]:
# Same but with GIN_Mean

In [5]:
atlas_labels_map6 = "atlas_gin_map6.nii.gz"
atlas_labels_svbp = "atlas_gin_svbp.nii.gz"

deep_labels = sorted(glob(os.path.join('output', 'GIN', 'baseline-ddf', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))
mean_labels = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', "*_deformable_convert.nii.gz")))

#dicemetric = monai.metrics.DiceMetric(num_classes=11)
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
#dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
#dice_metric(y_pred=val_outputs, y=val_labels)

values_deep = [0,0,0,0,0,0,0,0,0,0,0]
values_ants = [0,0,0,0,0,0,0,0,0,0,0]
values_mean = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
map6 = torch.from_numpy(nib.load(atlas_labels_map6).get_fdata().ravel()).type('torch.LongTensor')
count = 0

for i, _ in enumerate(ants_labels):  
    if "/M" in ants_labels[i]:
        count += 1
        print(ants_labels[i])
        print(deep_labels[i])
        deep = torch.from_numpy(nib.load(deep_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        deep[np.where(deep > 10)] = 0
        ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        ants[np.where(ants > 10)] = 0  
        mean = torch.from_numpy(nib.load(mean_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        mean[np.where(mean > 10)] = 0  

        deep_values = dicemetric(deep,map6)
        ants_values = dicemetric(ants,map6)    
        mean_values = dicemetric(mean,map6)  
        for j in range(len(deep_values)):
            values_deep[j] += deep_values[j].item()
            values_ants[j] += ants_values[j].item()
            values_mean[j] += mean_values[j].item()
        print(deep_values)
        print(ants_values)
        print(mean_values)
        print('-'*10)

for j in range(len(deep_values)):
    values_deep[j] /= count
    values_ants[j] /= count
    values_mean[j] /= count
    


print(values_deep)
print(values_ants)
print(values_mean)
print('-'*20) 
    

dataset3/GIN/Labels/M672_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-ddf/Labels/M672_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9878, 0.8372, 0.7997, 0.8865, 0.9040, 0.8947, 0.6540, 0.8870, 0.9121,
        0.7184, 0.8190])
tensor([0.9869, 0.7792, 0.7126, 0.8377, 0.8890, 0.8797, 0.4734, 0.8161, 0.8996,
        0.7520, 0.8078])
tensor([0.9880, 0.8394, 0.7753, 0.9044, 0.9034, 0.9087, 0.5899, 0.8902, 0.9212,
        0.6704, 0.7837])
----------
dataset3/GIN/Labels/M757_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-ddf/Labels/M757_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9882, 0.8311, 0.7763, 0.9036, 0.9055, 0.9065, 0.7028, 0.8966, 0.9200,
        0.6850, 0.8286])
tensor([0.9875, 0.7843, 0.5554, 0.8320, 0.8782, 0.8977, 0.5083, 0.8310, 0.9064,
        0.7537, 0.8297])
tensor([0.9886, 0.8247, 0.7730, 0.9029, 0.8994, 0.9158, 0.6320, 0.9043, 0.9222,
        0.6825, 0.8098])
----------
dataset3/GIN/Labels/M857_N4_o_128_

In [9]:
for i, name in enumerate(map6_names):
    print((name + ": ants={:.4f} | mean={:.4f} | deep={:.4f}").format(values_ants[i], values_mean[i], values_deep[i]))

bg: ants=0.9860 | mean=0.9877 | deep=0.9875
hy: ants=0.7432 | mean=0.8112 | deep=0.8133
mb: ants=0.5655 | mean=0.6807 | deep=0.6846
hpf: ants=0.8328 | mean=0.9062 | deep=0.9008
str: ants=0.8776 | mean=0.8969 | deep=0.9009
ctx: ants=0.8808 | mean=0.9082 | deep=0.8971
gp: ants=0.4671 | mean=0.5797 | deep=0.6464
th: ants=0.8152 | mean=0.8826 | deep=0.8794
cb: ants=0.8966 | mean=0.9196 | deep=0.9142
olf: ants=0.7266 | mean=0.6661 | deep=0.6904
bs: ants=0.7972 | mean=0.8073 | deep=0.8277


In [20]:
# Comparing with MajorityVoting affine

In [21]:
atlas_labels_map6 = "dataset3/Atlas/GIN_LabelsFusion_MajorityVoting_affine.nii.gz"

deep_labels = sorted(glob(os.path.join('output', 'GIN', 'test_convert', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))

#dicemetric = monai.metrics.DiceMetric(num_classes=11)
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
#dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
#dice_metric(y_pred=val_outputs, y=val_labels)

values_deep = [0,0,0,0,0,0,0,0,0,0,0]
values_ants = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
map6 = torch.from_numpy(nib.load(atlas_labels_map6).get_fdata().ravel()).type('torch.LongTensor')
count = 0

for i, _ in enumerate(ants_labels):  
    count += 1
    print(ants_labels[i])
    print(deep_labels[i])
    deep = torch.from_numpy(nib.load(deep_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep[np.where(deep > 10)] = 0
    ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ants[np.where(ants > 10)] = 0   
    
    deep_values = dicemetric(deep,map6)
    ants_values = dicemetric(ants,map6)    
    for j in range(len(deep_values)):
        values_deep[j] += deep_values[j].item()
        values_ants[j] += ants_values[j].item()
    print(deep_values)
    print(ants_values)
    print('-'*10)

for j in range(len(deep_values)):
    values_deep[j] /= count
    values_ants[j] /= count
    


print(values_deep)
print(values_ants)
print('-'*20) 
    

dataset3/GIN/Labels/653_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/653_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9918, 0.8945, 0.8163, 0.9112, 0.8748, 0.9180, 0.5818, 0.9011, 0.8750,
        0.8891, 0.0000])
tensor([0.9897, 0.7703, 0.5762, 0.8525, 0.8898, 0.9188, 0.6973, 0.8120, 0.8587,
        0.7107, 0.0000])
----------
dataset3/GIN/Labels/655_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/655_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9913, 0.8659, 0.8679, 0.9184, 0.8262, 0.9080, 0.5283, 0.9138, 0.8818,
        0.8799, 0.0000])
tensor([0.9878, 0.7677, 0.5369, 0.8493, 0.8394, 0.8895, 0.6561, 0.7778, 0.8569,
        0.6539, 0.0000])
----------
dataset3/GIN/Labels/656_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/656_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9921, 0.8977, 0.8650, 0.9127, 0.8879, 0.9212, 0.5542, 0.9214, 0.8780,
        0.908

tensor([9.8510e-01, 8.9768e-01, 8.3224e-01, 9.2076e-01, 8.8536e-01, 9.0414e-01,
        7.3635e-01, 9.1614e-01, 8.8297e-01, 8.5728e-01, 4.9517e-04])
tensor([9.8454e-01, 8.3237e-01, 7.8921e-01, 8.4406e-01, 8.8511e-01, 9.0294e-01,
        6.4491e-01, 8.6062e-01, 8.7810e-01, 6.6486e-01, 3.3514e-04])
----------
dataset3/GIN/Labels/M874_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/M874_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([9.8560e-01, 8.6607e-01, 8.2952e-01, 9.0501e-01, 8.5072e-01, 9.0734e-01,
        5.4688e-01, 8.7261e-01, 8.7683e-01, 8.7590e-01, 6.0071e-04])
tensor([9.8498e-01, 7.9355e-01, 8.1495e-01, 8.4600e-01, 8.4927e-01, 9.0390e-01,
        6.2424e-01, 8.3521e-01, 8.7523e-01, 7.3375e-01, 2.5571e-04])
----------
dataset3/GIN/Labels/M875_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/M875_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([9.8513e-01, 8.8182e-01, 7.9428e-01, 9.1112e-01, 8.8708e-01, 

In [22]:
for i, name in enumerate(map6_names):
    print((name + ": ants={:.4f} | deep={:.4f}").format(values_ants[i], values_deep[i]))

bg: ants=0.9872 | deep=0.9888
hypothalamus: ants=0.7789 | deep=0.8798
coliculi: ants=0.6326 | deep=0.8031
hippocampal formation: ants=0.8504 | deep=0.9076
striatum: ants=0.8691 | deep=0.8613
cortex: ants=0.9029 | deep=0.9089
globus pallidus: ants=0.6492 | deep=0.5747
thalamus: ants=0.8292 | deep=0.8888
cerebelum: ants=0.8665 | deep=0.8756
olfactory bulb: ants=0.6891 | deep=0.8528
brainstem: ants=0.0001 | deep=0.0002


In [ ]:
# Comparing with MajorityVoting deformable

In [23]:
atlas_labels_map6 = "dataset3/Atlas/GIN_LabelsFusion_MajorityVoting_deformable.nii.gz"

deep_labels = sorted(glob(os.path.join('output', 'GIN', 'test_convert', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))

#dicemetric = monai.metrics.DiceMetric(num_classes=11)
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
#dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
#dice_metric(y_pred=val_outputs, y=val_labels)

values_deep = [0,0,0,0,0,0,0,0,0,0,0]
values_ants = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
map6 = torch.from_numpy(nib.load(atlas_labels_map6).get_fdata().ravel()).type('torch.LongTensor')
count = 0

for i, _ in enumerate(ants_labels):  
    count += 1
    print(ants_labels[i])
    print(deep_labels[i])
    deep = torch.from_numpy(nib.load(deep_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep[np.where(deep > 10)] = 0
    ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ants[np.where(ants > 10)] = 0   
    
    deep_values = dicemetric(deep,map6)
    ants_values = dicemetric(ants,map6)    
    for j in range(len(deep_values)):
        values_deep[j] += deep_values[j].item()
        values_ants[j] += ants_values[j].item()
    print(deep_values)
    print(ants_values)
    print('-'*10)

for j in range(len(deep_values)):
    values_deep[j] /= count
    values_ants[j] /= count
    


print(values_deep)
print(values_ants)
print('-'*20)      

dataset3/GIN/Labels/653_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/653_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9891, 0.7647, 0.5553, 0.8550, 0.8797, 0.9065, 0.6745, 0.8004, 0.8983,
        0.7231, 0.0000])
tensor([0.9955, 0.8946, 0.8402, 0.9540, 0.9321, 0.9587, 0.8597, 0.9136, 0.9317,
        0.9155, 0.0000])
----------
dataset3/GIN/Labels/655_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/655_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9898, 0.7300, 0.6461, 0.8580, 0.8318, 0.9081, 0.5639, 0.8301, 0.8995,
        0.7368, 0.0000])
tensor([0.9943, 0.8649, 0.8229, 0.9351, 0.8638, 0.9457, 0.7411, 0.8777, 0.9373,
        0.8943, 0.0000])
----------
dataset3/GIN/Labels/656_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/656_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9896, 0.7792, 0.6209, 0.8495, 0.8883, 0.9075, 0.5985, 0.8371, 0.9014,
        0.735

tensor([9.8300e-01, 7.8743e-01, 5.7286e-01, 8.3116e-01, 8.7535e-01, 8.9119e-01,
        6.3228e-01, 7.9543e-01, 8.7188e-01, 6.5589e-01, 7.2072e-04])
tensor([0.9888, 0.8832, 0.7352, 0.9299, 0.8997, 0.9451, 0.7800, 0.8693, 0.9191,
        0.8427, 0.0012])
----------
dataset3/GIN/Labels/M875_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/M875_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([9.8314e-01, 8.1996e-01, 5.5392e-01, 8.5527e-01, 8.8816e-01, 8.9159e-01,
        7.1293e-01, 8.4304e-01, 8.6323e-01, 6.9870e-01, 6.3542e-04])
tensor([0.9884, 0.8763, 0.6654, 0.9219, 0.9173, 0.9414, 0.8129, 0.8807, 0.9088,
        0.8692, 0.0011])
----------
dataset3/GIN/Labels/M877_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/test_convert/Labels/M877_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([9.8398e-01, 8.1924e-01, 0.0000e+00, 8.2911e-01, 8.8287e-01, 9.0564e-01,
        7.1996e-01, 8.4822e-01, 8.7713e-01, 6.6834e-01, 4.8077e-04])
tensor(

In [24]:
for i, name in enumerate(map6_names):
    print((name + ": ants={:.4f} | deep={:.4f}").format(values_ants[i], values_deep[i]))    

bg: ants=0.9926 | deep=0.9868
hypothalamus: ants=0.8660 | deep=0.7692
coliculi: ants=0.7867 | deep=0.5665
hippocampal formation: ants=0.9350 | deep=0.8381
striatum: ants=0.9081 | deep=0.8707
cortex: ants=0.9480 | deep=0.9000
globus pallidus: ants=0.7669 | deep=0.6252
thalamus: ants=0.8894 | deep=0.7961
cerebelum: ants=0.9292 | deep=0.8854
olfactory bulb: ants=0.8698 | deep=0.7097
brainstem: ants=0.0005 | deep=0.0003


In [5]:
outfolders = ["baseline-sym-ddf",
              "baseline-sym-noise", 
              
              "baseline-ddf",
              "baseline-noise",
               
              "baseline",
              "baseline-sym"
             ]

In [8]:
for outfolder in outfolders:
    atlas_labels_map6 = "atlas_gin_map6.nii.gz"
    atlas_labels_svbp = "atlas_gin_svbp.nii.gz"

    deep_labels = sorted(glob(os.path.join('output', 'GIN', outfolder, 'Labels', '*.nii.gz')))
    ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))

    #dicemetric = monai.metrics.DiceMetric(num_classes=11)
    dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)
    #dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
    #dice_metric(y_pred=val_outputs, y=val_labels)

    values_deep = [0,0,0,0,0,0,0,0,0,0,0]
    values_ants = [0,0,0,0,0,0,0,0,0,0,0]

    map6 = torch.from_numpy(nib.load(atlas_labels_map6).get_fdata().ravel()).type('torch.LongTensor')
    count = 0

    for i, _ in enumerate(ants_labels):  
        count += 1
        print(ants_labels[i])
        print(deep_labels[i])
        deep = torch.from_numpy(nib.load(deep_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        deep[np.where(deep > 10)] = 0
        ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        ants[np.where(ants > 10)] = 0   

        deep_values = dicemetric(deep,map6)
        ants_values = dicemetric(ants,map6)    
        for j in range(len(deep_values)):
            values_deep[j] += deep_values[j].item()
            values_ants[j] += ants_values[j].item()
        print(deep_values)
        print(ants_values)
        print('-'*10)

    for j in range(len(deep_values)):
        values_deep[j] /= count
        values_ants[j] /= count



    print(values_deep)
    print(values_ants)
    print('-'*20) 

    for i, name in enumerate(map6_names):
        print((name + ": ants={:.4f} | deep={:.4f}").format(values_ants[i], values_deep[i]))
    print('*'*20)
    print('*'*20)
    print('*'*20)

dataset3/GIN/Labels/653_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-ddf/Labels/653_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9850, 0.8416, 0.7665, 0.9039, 0.8868, 0.9157, 0.7236, 0.8950, 0.9043,
        0.6962, 0.0000])
tensor([0.9815, 0.7265, 0.4498, 0.8033, 0.8616, 0.8972, 0.6043, 0.7482, 0.8318,
        0.6327, 0.0000])
----------
dataset3/GIN/Labels/655_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-ddf/Labels/655_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9839, 0.8270, 0.7400, 0.8893, 0.8178, 0.9122, 0.6680, 0.8533, 0.9023,
        0.6532, 0.0000])
tensor([0.9799, 0.7160, 0.4448, 0.8061, 0.8034, 0.8765, 0.6888, 0.7187, 0.8259,
        0.6218, 0.0000])
----------
dataset3/GIN/Labels/656_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-ddf/Labels/656_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9846, 0.8414, 0.7596, 0.8832, 0.8781, 0.9163, 0.7405, 0.8831, 0.8955,
 

tensor([0.9883, 0.8157, 0.7864, 0.9083, 0.9147, 0.9043, 0.7190, 0.8640, 0.9229,
        0.6220, 0.8374])
tensor([0.9872, 0.7444, 0.6906, 0.8203, 0.8837, 0.8849, 0.6465, 0.7972, 0.8990,
        0.7823, 0.8075])
----------
dataset3/GIN/Labels/M875_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-ddf/Labels/M875_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9869, 0.8065, 0.7216, 0.8991, 0.8910, 0.8810, 0.5652, 0.8737, 0.9202,
        0.6604, 0.8496])
tensor([0.9871, 0.7353, 0.6605, 0.8598, 0.8926, 0.8954, 0.5502, 0.8324, 0.9154,
        0.7222, 0.8258])
----------
dataset3/GIN/Labels/M877_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-ddf/Labels/M877_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9868, 0.8211, 0.0000, 0.9111, 0.9015, 0.9052, 0.5897, 0.8727, 0.9151,
        0.6543, 0.8399])
tensor([0.9852, 0.7746, 0.0000, 0.8318, 0.8721, 0.8856, 0.4047, 0.8212, 0.8947,
        0.7082, 0.7860])
----------
dataset3/GIN/La

tensor([0.9302, 0.2523, 0.0000, 0.3404, 0.2175, 0.3638, 0.0000, 0.0000, 0.1640,
        0.1410, 0.0000])
tensor([0.9786, 0.5331, 0.4703, 0.7346, 0.7967, 0.8726, 0.5344, 0.7100, 0.7959,
        0.6273, 0.0000])
----------
dataset3/GIN/Labels/683_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-noise/Labels/683_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9281, 0.2464, 0.0000, 0.3535, 0.2674, 0.3594, 0.0053, 0.0209, 0.2256,
        0.1320, 0.0000])
tensor([0.9814, 0.7294, 0.4733, 0.7964, 0.8467, 0.8899, 0.6571, 0.7309, 0.8059,
        0.7032, 0.0000])
----------
dataset3/GIN/Labels/M672_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym-noise/Labels/M672_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9287, 0.2575, 0.0000, 0.3826, 0.2410, 0.3530, 0.0000, 0.0000, 0.2372,
        0.1452, 0.3756])
tensor([0.9869, 0.7792, 0.7126, 0.8377, 0.8890, 0.8797, 0.4734, 0.8161, 0.8996,
        0.7520, 0.8078])
----------
dataset3/GIN/

tensor([0.9841, 0.8351, 0.7864, 0.8847, 0.8916, 0.9042, 0.7331, 0.8762, 0.8823,
        0.7432, 0.0000])
tensor([0.9815, 0.7350, 0.4566, 0.7869, 0.8720, 0.8969, 0.6632, 0.7196, 0.8321,
        0.6559, 0.0000])
----------
dataset3/GIN/Labels/665_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-ddf/Labels/665_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9839, 0.7890, 0.7838, 0.8623, 0.8791, 0.8956, 0.6705, 0.8533, 0.9010,
        0.6412, 0.0000])
tensor([0.9805, 0.6733, 0.5522, 0.7690, 0.8542, 0.8722, 0.5657, 0.7793, 0.8300,
        0.6115, 0.0000])
----------
dataset3/GIN/Labels/667_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-ddf/Labels/667_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9845, 0.8164, 0.8075, 0.8804, 0.8788, 0.9130, 0.6613, 0.8528, 0.8926,
        0.7134, 0.0000])
tensor([0.9810, 0.7326, 0.6192, 0.8053, 0.8736, 0.8765, 0.5852, 0.8251, 0.8252,
        0.6187, 0.0000])
----------
dataset3/GIN/Labels/668_N4_

tensor([0.9823, 0.8056, 0.7539, 0.9014, 0.8846, 0.8549, 0.6816, 0.8816, 0.8794,
        0.6075, 0.0000])
tensor([0.9815, 0.7265, 0.4498, 0.8033, 0.8616, 0.8972, 0.6043, 0.7482, 0.8318,
        0.6327, 0.0000])
----------
dataset3/GIN/Labels/655_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-noise/Labels/655_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9805, 0.8061, 0.7190, 0.8913, 0.7977, 0.8293, 0.5809, 0.8472, 0.8791,
        0.5998, 0.0000])
tensor([0.9799, 0.7160, 0.4448, 0.8061, 0.8034, 0.8765, 0.6888, 0.7187, 0.8259,
        0.6218, 0.0000])
----------
dataset3/GIN/Labels/656_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-noise/Labels/656_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9812, 0.8127, 0.7600, 0.8878, 0.8688, 0.8389, 0.6651, 0.8726, 0.8688,
        0.5914, 0.0000])
tensor([0.9802, 0.6899, 0.4426, 0.7898, 0.8291, 0.8829, 0.6638, 0.7013, 0.8354,
        0.5826, 0.0000])
----------
dataset3/GIN/Labels/660

tensor([0.9855, 0.7976, 0.7559, 0.9102, 0.9052, 0.8414, 0.6568, 0.8611, 0.9092,
        0.5545, 0.8071])
tensor([0.9872, 0.7444, 0.6906, 0.8203, 0.8837, 0.8849, 0.6465, 0.7972, 0.8990,
        0.7823, 0.8075])
----------
dataset3/GIN/Labels/M875_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-noise/Labels/M875_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9851, 0.7664, 0.6990, 0.9071, 0.8933, 0.8481, 0.4895, 0.8590, 0.9055,
        0.6173, 0.8176])
tensor([0.9871, 0.7353, 0.6605, 0.8598, 0.8926, 0.8954, 0.5502, 0.8324, 0.9154,
        0.7222, 0.8258])
----------
dataset3/GIN/Labels/M877_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-noise/Labels/M877_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9838, 0.7893, 0.0000, 0.9098, 0.8809, 0.8363, 0.4650, 0.8594, 0.8931,
        0.6002, 0.8042])
tensor([0.9852, 0.7746, 0.0000, 0.8318, 0.8721, 0.8856, 0.4047, 0.8212, 0.8947,
        0.7082, 0.7860])
----------
dataset3/GIN/Labels

tensor([0.8946, 0.0000, 0.0000, 0.0000, 0.0000, 0.2958, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
tensor([0.9814, 0.7294, 0.4733, 0.7964, 0.8467, 0.8899, 0.6571, 0.7309, 0.8059,
        0.7032, 0.0000])
----------
dataset3/GIN/Labels/M672_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline/Labels/M672_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9025, 0.0000, 0.0000, 0.0000, 0.0000, 0.3117, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
tensor([0.9869, 0.7792, 0.7126, 0.8377, 0.8890, 0.8797, 0.4734, 0.8161, 0.8996,
        0.7520, 0.8078])
----------
dataset3/GIN/Labels/M757_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline/Labels/M757_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.8947, 0.0000, 0.0000, 0.0000, 0.0000, 0.2973, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
tensor([0.9875, 0.7843, 0.5554, 0.8320, 0.8782, 0.8977, 0.5083, 0.8310, 0.9064,
        0.7537, 0.8297])
----------
dataset3/GIN/Labels/M857_N4_o_1

tensor([0.9802, 0.7688, 0.7644, 0.8791, 0.8173, 0.8248, 0.4964, 0.8519, 0.8703,
        0.6579, 0.0000])
tensor([0.9805, 0.6733, 0.5522, 0.7690, 0.8542, 0.8722, 0.5657, 0.7793, 0.8300,
        0.6115, 0.0000])
----------
dataset3/GIN/Labels/667_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym/Labels/667_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9813, 0.7922, 0.8089, 0.8828, 0.8443, 0.8449, 0.5378, 0.8618, 0.8757,
        0.6992, 0.0000])
tensor([0.9810, 0.7326, 0.6192, 0.8053, 0.8736, 0.8765, 0.5852, 0.8251, 0.8252,
        0.6187, 0.0000])
----------
dataset3/GIN/Labels/668_N4_o_128_ras_norm_id_affine_deformable_convert.nii.gz
output/GIN/baseline-sym/Labels/668_N4_o_128_ras_norm_id_affine_labels.nii.gz
tensor([0.9811, 0.7948, 0.7589, 0.8928, 0.8358, 0.8507, 0.4604, 0.8595, 0.8789,
        0.6605, 0.0000])
tensor([0.9806, 0.7201, 0.5189, 0.8004, 0.8482, 0.8867, 0.5761, 0.7543, 0.8300,
        0.6342, 0.0000])
----------
dataset3/GIN/Labels/670_N4_